In [1]:
#widen display #thanks Nick!
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))

# Project Description:
## Compare local property values against the availability and quality of surrounding businesses.

### Team Members:
Amy, Ebony, Jin, Michael, Syed

### Datasets:
Dallas County Appraisals, Google Maps API

### Questions:
   1.    Is there a correlation between housing prices and the commercial businesses that are available in that area? (comparing high housing price area vs low housing price)
   2.    How does the property value compare with the availability of different types of businesses in the area?
          -  Banks
          -  Grocery Stores
          -  Hospitals
          -  Cafes (Starbucks)
   3.     Where in the Dallas area are the highest and lowest property values?

In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json
import gmaps
import time

# API Keys
from config import api_key

### Gather and clean data from Dallas County Appraisal District (DCAD) which contains the address and value for all properties in Dallas County (~500k records)

In [3]:
# Read in the Dallas County Appraisal District (DCAD) property values file
file = "Resources/dcad_combined.csv"

#create DataFrame using only the columns we need
dcad_df = pd.read_csv(file, usecols=['PROPERTY_ZIPCODE', 'TOT_VAL'])
dcad_df.head(5)

,PROPERTY_ZIPCODE,TOT_VAL
0,750513060,103500
1,750513040,145500
2,750502208,168040
3,750502277,200040
4,750617840,151880


In [4]:
#add column for 5 digit zipcode
dcad_df['ZIPCODE'] = dcad_df['PROPERTY_ZIPCODE'].astype(str).str[:5]
dcad_df.head(5)

,PROPERTY_ZIPCODE,TOT_VAL,ZIPCODE
0,750513060,103500,75051
1,750513040,145500,75051
2,750502208,168040,75050
3,750502277,200040,75050
4,750617840,151880,75061


In [5]:
#Group Property Values by Zipcode
zip_group = dcad_df.groupby('ZIPCODE')['TOT_VAL']
zip_group.mean().head().map("${:,.2f}".format)

ZIPCODE
75001    $315,208.49
75006    $202,706.61
75007    $200,411.36
75019    $415,797.68
75038    $358,436.94
Name: TOT_VAL, dtype: object

### Create Dataframe from groupby with proprty count mean value, rename columns, and sort

In [6]:
zip_group_df = dcad_df.groupby('ZIPCODE')['TOT_VAL'].agg(('count', 'mean'))\
                    .rename(columns={'count':'PropertyValueCount', 'mean':'MeanPropertyValue'})\
                    .sort_values(by='MeanPropertyValue', ascending=False)\
                    .reset_index()

### Add columns to DataFrame to store the business data we are going to merge

In [7]:
zip_group_df["MeanPropertyDollarValue"] = zip_group_df["MeanPropertyValue"].map("${:,.0f}".format)
zip_group_df['Lat'] = "" 
zip_group_df["Lng"] = ""
zip_group_df["City"] = ""
zip_group_df["State"] = ""
zip_group_df = zip_group_df.rename(columns={"ZIPCODE": "Zipcode"})

In [8]:
zip_group_df.head()

,Zipcode,PropertyValueCount,MeanPropertyValue,MeanPropertyDollarValue,Lat,Lng,City,State
0,75205,6415,1.606362e+06,"$1,606,362",,,,
1,75225,7633,1.207824e+06,"$1,207,824",,,,
2,75201,1326,9.667580e+05,"$966,758",,,,
3,75209,5424,7.230552e+05,"$723,055",,,,
4,75230,9337,7.046378e+05,"$704,638",,,,


## Gather data from Google Maps API about businesses by Zipcode
1. Geocode each Zipcode from DCAD
2. Use lat/lng to search for businesses in each Zipcode
3. Merge with property data which will be used to create charts

### Geocding: loop through each zipcode and update the lat/lng in our dataframe

In [9]:
# create a params dict that will be updated with new zipcode each iteration
params = {"key": api_key}

# Loop through the zipcode pd's and run a lat/long search for each
for index, row in zip_group_df.iterrows():
    base_geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['Zipcode']

    # update address key value to zipcode
    params['address'] = zipcode

    # make request
    zips_lat_lng = requests.get(base_geocode_url, params=params)
    
    # convert to json
    zips_lat_lng = zips_lat_lng.json()
    #print(json.dumps(zips_lat_lng, indent=4, sort_keys=True))
    
    #loop through address to find city
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'locality':
            zip_group_df.loc[index, "City"] = i['long_name']
    
    #loop though address to find state
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'administrative_area_level_1':
            zip_group_df.loc[index, "State"] = i['short_name']
    
    #Update Lat, Long in Dataframe
    zip_group_df.loc[index, "Lat"] = zips_lat_lng["results"][0]["geometry"]["location"]["lat"]
    zip_group_df.loc[index, "Lng"] = zips_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [10]:
#visualize to lat/lng have been populated
zip_group_df.head()

,Zipcode,PropertyValueCount,MeanPropertyValue,MeanPropertyDollarValue,Lat,Lng,City,State
0,75205,6415,1.606362e+06,"$1,606,362",32.8326,-96.7976,Dallas,TX
1,75225,7633,1.207824e+06,"$1,207,824",32.8695,-96.7896,Dallas,TX
2,75201,1326,9.667580e+05,"$966,758",32.7863,-96.7963,Dallas,TX
3,75209,5424,7.230552e+05,"$723,055",32.8539,-96.819,Dallas,TX
4,75230,9337,7.046378e+05,"$704,638",32.9005,-96.7869,Dallas,TX


### Function for calling Google Places search and paging through the results

In [11]:
def findPlaces(params):
    #set global variable so name is returned
    global next_page_token
    global results_list

    #Google Places Search
    base_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_places_url, params=params)
    
    search_results = json.loads(response.text)
    results_list = search_results["results"]
    
    #Set next page token. Default is None.
    next_page_token = search_results.get("next_page_token", None)
    
    time.sleep(4) #add delay because it can take a moment before the the page token is actually available to make the subsequent call

    return next_page_token, results_list

### Define the types of businesses to lookup:  we chose to focus on essential businesses like banks, supermarkets, hospitals and starbucks

In [12]:
#Google Places Types: https://developers.google.com/places/web-service/supported_types

#The types list can include multiple categories. A count and mean rating will be added to the final DataFrame for each type
types = ['bank','supermarket','hospital', 'cafe']

bank_names = ['bank', 'credit union']
not_supermarket_names = ['dollar', 'cents', 'liquor', 'convenience']
hospital_names = ['hospital', 'medical center', 'parkland']
cafe_names = ['starbucks']

### Count and summarize ratings for each business type using the lat/lng by radius using Google Places API and update our dataframe

In [13]:
# params dictionary to update each iteration
for each_type in types:
    
    #clear console before starting a new type
    clear_output()
    
    params = {
        #3 mi radius. A Zipcode is not returned in the results, so we cannot 
        #match against our zipcode without doing a reverse lookup for every result
        "radius": 4828,
        "types": each_type,
        "key": api_key,
    }
    
    if each_type == 'cafe':
        params["keyword"] = 'starbucks' 
    else: 
        params["keyword"] = None
    
    #variables for the specific column names for the business type we are searching
    count_column = f"{each_type}_count"
    rating_column = f"{each_type}_rating"
    
    #add columns for the business type we are looking up
    zip_group_df[count_column] = 0
    zip_group_df[rating_column] = 0

   
    # Use the lat/lng we recovered to search for businesses
    for index, row in zip_group_df.iterrows():

        rating_sum = 0
        rating_count = 0
        business_count = 0

        next_page_token = "" #initialize the page token

        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

        # change location each iteration while leaving original params in place
        #params["location"] = f"32.8326,-96.7976" #testing 1 lat, lng
        params["location"] = f"{lat},{lng}"

        while next_page_token != None:
            
            params["pagetoken"] = next_page_token
            
            #call places function
            findPlaces(params)

            #Loop through results list to count each business and get rating for each 
            for each_result in results_list:
                
                #get name in lower case
                business_name_lower = each_result["name"].lower()

                #check to make sure a Rating since not all business have a rating
                if each_result.get("rating") != None:
                    
                    #check for names and keywords based on business type to exclude businesses that are not a good match
                    if each_type == "bank" and any(n in business_name_lower for n in bank_names):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating                        
                    
                    if each_type == "hospital" and any(n in business_name_lower for n in hospital_names):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating     
                    
                    if each_type == "supermarket" and not(any(n in business_name_lower for n in not_supermarket_names)):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating
                        
                    if each_type == "cafe" and any(n in business_name_lower for n in cafe_names):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating                             
                                
                    #for printing to console
                    info = f'{row["Zipcode"]} | {each_result["name"]} | location: ({round(each_result["geometry"]["location"]["lat"],2)}, {round(each_result["geometry"]["location"]["lng"],2)}) | rating: {each_result.get("rating",0)}'
                    print(info)
            
            #Set Mean Rating to None if no businesses found
            try:
                mean_rating = rating_sum / business_count
                zip_group_df.loc[index, rating_column] = mean_rating
            except(ZeroDivisionError):
                zip_group_df.loc[index, rating_column] = None
                
            #Set business count in dataframe
            zip_group_df.loc[index, count_column] = business_count
            
        print(f'\nTotal number of {each_type}s found for {row["Zipcode"]}: {business_count} | rating_sum: {round(rating_sum,2)} | avg_rating: {round(mean_rating,2)}')
        print(f'------------------------------------------------------------------------------------------------------\n')

75205 | Hillcrest Bank | location: (32.8, -96.81) | rating: 3.7
75205 | Comerica Bank | location: (32.85, -96.77) | rating: 3.2
75205 | BBVA Bank | location: (32.86, -96.81) | rating: 4.2
75205 | Bank of America Financial Center | location: (32.81, -96.81) | rating: 2.1
75205 | Amegy Bank | location: (32.83, -96.78) | rating: 2.3
75205 | PlainsCapital Bank | location: (32.8, -96.81) | rating: 4.5
75205 | Wells Fargo Bank | location: (32.8, -96.81) | rating: 3.9
75205 | Wells Fargo Bank | location: (32.84, -96.75) | rating: 4.4
75205 | Wells Fargo Bank | location: (32.85, -96.81) | rating: 4
75205 | First Citizens Bank | location: (32.8, -96.8) | rating: 3.4
75205 | Chase Bank | location: (32.82, -96.81) | rating: 1.8
75205 | Comerica Bank | location: (32.86, -96.8) | rating: 5
75205 | Chase Bank | location: (32.81, -96.8) | rating: 3.9
75205 | Bank of America Financial Center | location: (32.83, -96.83) | rating: 3
75205 | BBVA Bank | location: (32.8, -96.81) | rating: 3.6
75205 | Chas

75209 | Chase Bank | location: (32.86, -96.84) | rating: 4.3
75209 | BBVA Bank | location: (32.86, -96.81) | rating: 4.2
75209 | Wells Fargo Bank | location: (32.85, -96.81) | rating: 4
75209 | Comerica Bank | location: (32.86, -96.8) | rating: 5
75209 | Comerica Bank | location: (32.85, -96.77) | rating: 3.2
75209 | Chase Bank | location: (32.82, -96.81) | rating: 1.8
75209 | Amegy Bank | location: (32.83, -96.78) | rating: 2.3
75209 | Regions Bank | location: (32.87, -96.81) | rating: 2.4
75209 | Frost Bank | location: (32.86, -96.8) | rating: 4.5
75209 | Bank of America Financial Center | location: (32.83, -96.83) | rating: 3
75209 | Inwood National Bank | location: (32.85, -96.82) | rating: 3.9
75209 | Bank of America Financial Center | location: (32.81, -96.81) | rating: 2.1
75209 | Cadence Trust | location: (32.86, -96.81) | rating: 5
75209 | Bank of Texas | location: (32.84, -96.79) | rating: 4.3
75209 | Bank of Texas | location: (32.85, -96.8) | rating: 4.4
75209 | Chase Bank |

75229 | Wells Fargo Bank | location: (32.91, -96.86) | rating: 3
75229 | Chase Bank | location: (32.88, -96.88) | rating: 2.3
75229 | Bank of America Financial Center | location: (32.9, -96.87) | rating: 2.1
75229 | Chase Bank | location: (32.91, -96.86) | rating: 2.2
75229 | BBVA Bank | location: (32.86, -96.87) | rating: 4.2
75229 | American First National Bank | location: (32.89, -96.89) | rating: 1.6
75229 | Shinhan Bank America | location: (32.9, -96.91) | rating: 4.5
75229 | American Bank | location: (32.86, -96.88) | rating: 3.6
75229 | Comerica Bank | location: (32.91, -96.87) | rating: 2.3
75229 | State Bank of Texas | location: (32.91, -96.87) | rating: 3
75229 | BB&T | location: (32.91, -96.86) | rating: 1.4
75229 | Shinhan Bank America Texas Loan Center | location: (32.9, -96.9) | rating: 5
75229 | Commonwealth Business Bank (CBB Bank) | location: (32.9, -96.91) | rating: 5
75229 | First Convenience Bank | location: (32.92, -96.84) | rating: 2.4
75229 | Capital One Bank | l

75039 | Origin Bank | location: (32.91, -96.96) | rating: 5
75039 | Southside Bank | location: (32.89, -96.97) | rating: 2.6
75039 | Simmons Bank | location: (32.91, -96.92) | rating: 3
75039 | Credit Union of Texas | location: (32.89, -96.98) | rating: 3.4
75039 | First Convenience Bank | location: (32.92, -96.97) | rating: 1.8
75039 | ATM Wal-mart | location: (32.92, -96.97) | rating: 4
75039 | Chase Bank | location: (32.86, -96.93) | rating: 4.2
75039 | Commonwealth Business Bank (CBB Bank) | location: (32.9, -96.91) | rating: 5
75039 | Shinhan Bank America Texas Loan Center | location: (32.9, -96.9) | rating: 5
75039 | BBVA Bank | location: (32.87, -96.95) | rating: 3.7
75039 | Veritex Bank | location: (32.87, -96.99) | rating: 5
75039 | Bank of America Mortgage | location: (32.86, -96.93) | rating: 5
75039 | Chase Bank | location: (32.91, -96.96) | rating: 3.6
75039 | North Dallas Bank & Trust Co | location: (32.91, -96.96) | rating: 3.7
75039 | Bank of Hope | location: (32.9, -96

75248 | BB&T | location: (32.95, -96.8) | rating: 3
75248 | Chase Bank | location: (32.95, -96.77) | rating: 3.3
75248 | Veritex Community Bank | location: (32.98, -96.83) | rating: 4
75248 | Wells Fargo Bank | location: (32.96, -96.77) | rating: 4.3
75248 | BBVA Bank | location: (32.95, -96.8) | rating: 1.7
75248 | BBVA Bank | location: (32.99, -96.8) | rating: 3.2
75248 | BBVA Bank | location: (32.95, -96.77) | rating: 2.1
75248 | Liberty Capital Bank | location: (32.97, -96.82) | rating: 3.7
75248 | Prestige Community Credit Union | location: (32.96, -96.82) | rating: 4.3
75248 | BB&T | location: (33.0, -96.77) | rating: 4.8
75248 | Ocwen | location: (32.98, -96.83) | rating: 1
75248 | Bank of Texas | location: (33.0, -96.77) | rating: 4.8
75248 | North Dallas Bank & Trust Co | location: (32.97, -96.82) | rating: 4
75248 | Wells Fargo Bank | location: (32.96, -96.82) | rating: 2.1
75248 | Chase Bank | location: (33.0, -96.8) | rating: 2.4
75248 | Pavillion Bank | location: (32.98, -

75219 | Comerica Bank | location: (32.79, -96.8) | rating: 2.8
75219 | Comerica Bank | location: (32.82, -96.79) | rating: 3.6
75219 | Pegasus Bank | location: (32.84, -96.83) | rating: 5
75219 | ATM (7-Eleven) | location: (32.81, -96.81) | rating: 4
75219 | Belmont Wealth Management | location: (32.8, -96.81) | rating: 1
75219 | First Convenience Bank | location: (32.81, -96.81) | rating: 4.3
75219 | Veritex Bank | location: (32.81, -96.81) | rating: 3
75219 | IBC Bank | location: (32.8, -96.81) | rating: 4.3
75219 | Origin Bank | location: (32.81, -96.8) | rating: 5
75219 | Midwest Regional Bank | location: (32.8, -96.8) | rating: 3
75219 | BB&T | location: (32.8, -96.8) | rating: 2.6
75219 | Frost Bank | location: (32.8, -96.81) | rating: 4.3
75219 | Commerce Bank - Commercial Banking Office | location: (32.79, -96.81) | rating: 3.3

Total number of banks found for 75219: 39 | rating_sum: 142.1 | avg_rating: 3.64
----------------------------------------------------------------------

KeyboardInterrupt: 

In [ ]:
#create file to reference in visualizations (2_visualize-property-values-and-businesses.ipynb) so we don't have to make api calls during presentation
zip_group_df.to_csv('Resources/_zip_group_csv_beta.csv')

In [ ]:
zip_group_df.head()